In [2]:
import pandas as pd
import numpy as np


In [7]:
df=pd.read_csv("/workspaces/Demographic-Data-Analyzer/adult.data.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# 
How many people of each race are represented in this dataset?


In [8]:
set_race = {}
list_race  = []
for i in df['race']:
    list_race.append(i)
set_race = set(list_race)
set_race

{'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'}

In [9]:


# How many of each race are represented in this dataset? 
#This should be a Pandas series with race names as the index labels.
White = df[df['race'] == 'White']
Black = df[df['race'] == 'Black']
Amer_Indian_Eskimo = df[df['race'] == 'Amer-Indian-Eskimo']
Asian_Pac_Islander = df[df['race'] == 'Asian-Pac-Islander']
Other = df[df['race'] == 'Other']



In [10]:


list_race_population = [len(White), len(Black), len(Amer_Indian_Eskimo), len(Asian_Pac_Islander), len(Other)]
list_race_population



[27816, 3124, 311, 1039, 271]

In [11]:


race_series = pd.Series(data = list_race_population, index = set_race)
race_series



White                 27816
Amer-Indian-Eskimo     3124
Black                   311
Asian-Pac-Islander     1039
Other                   271
dtype: int64

# 
What is the average age of men?


In [12]:
men = df[df['sex'] == 'Male']
round(men['age'].mean(),1)

np.float64(39.4)

# What is the average age of wom

In [13]:


# What is the average age of women?
women = df[df['sex'] == 'Female']
round(women['age'].mean(),1)



np.float64(36.9)

# What is the percentage of people who have a Bachelor's degree?

In [14]:
bachelors_degree = df[df['education'] == 'Bachelors']
bachelors_degree['education']

0        Bachelors
1        Bachelors
4        Bachelors
9        Bachelors
11       Bachelors
           ...    
32530    Bachelors
32531    Bachelors
32533    Bachelors
32536    Bachelors
32538    Bachelors
Name: education, Length: 5355, dtype: object

In [15]:


# What is the percentage of people who don't have a Bachelor's degree?
not_bachelors_degree = df[df['education'] != 'Bachelors']
not_bachelors_degree['education']



2           HS-grad
3              11th
5           Masters
6               9th
7           HS-grad
            ...    
32556    Assoc-acdm
32557       HS-grad
32558       HS-grad
32559       HS-grad
32560       HS-grad
Name: education, Length: 27206, dtype: object

In [16]:
percantage_bachelor = round((len(bachelors_degree['education'])/len(df))*100,1)
percantage_bachelor

16.4

# 
What percentage of people with advanced education (Bachelors, Masters, or Doctorate) make more than 50K? What percentage of people without advanced education make more than 50K?


In [17]:


total_people = df.education.count()
number_of_people = total_people

number_of_people_with_adv_edu = len(df[((df.education == 'Bachelors') | (df.education == 'Masters') | (df.education == 'Doctorate'))])
number_of_people_with_low_edu = len(df[((df.education != 'Bachelors') & (df.education != 'Masters') & (df.education != 'Doctorate'))])

# with and without `Bachelors`, `Masters`, or `Doctorate`
higher_education = number_of_people_with_adv_edu
lower_education = number_of_people_with_low_edu

# percentage with salary >50K
people_with_adv_edu_high_income = (df[((df.education == 'Bachelors') | (df.education == 'Masters') | (df.education == 'Doctorate')) & (df.salary == '>50K')].count())['age']
higher_education_rich = round(((people_with_adv_edu_high_income / number_of_people_with_adv_edu) * 100), 1)

people_with_low_edu_high_income = (df[((df.education != 'Bachelors') & (df.education != 'Masters') & (df.education != 'Doctorate')) & (df.salary == '>50K')].count())['age']
lower_education_rich = round((people_with_low_edu_high_income / lower_education) * 100, 1)
print("People with high education and high income:", higher_education_rich)
print("Percentage of people with low education and high income:", lower_education_rich)



People with high education and high income: 46.5
Percentage of people with low education and high income: 17.4


# What country has the highest percentage of people that earn >50K and what is that percentag

In [18]:
#groups by native country and salary >50K, converts the series to dataframe and renames the columns
country = df.groupby(['native-country', df.salary[df.salary == '>50K']])['age'].size()
country = country.to_frame()
country.rename(columns={'age': 'filtered'}, inplace=True)

#groups by native country, coverts the series to dataframe and renames the columns
num_people_per_country = df.groupby(['native-country'])['age'].size()
num_people_per_country = num_people_per_country.to_frame()
num_people_per_country.rename(columns={'age': 'Total'}, inplace=True)

#merging the two dataframes
merged = country.merge(num_people_per_country, left_index=True, right_index=True)
merged['percentage'] = (merged.filtered / merged.Total) * 100

#calculates the percentages
max_percentage = merged['percentage'].max()
max_percentage_country = merged.index[merged['percentage']== max_percentage]

highest_earning_country = max_percentage_country[0][0]
highest_earning_country_percentage = round(max_percentage, 1)
print('Highest earning country:', highest_earning_country)
print("Highest earning country's percentage:", highest_earning_country_percentage)

Highest earning country: Iran
Highest earning country's percentage: 41.9


# What is the minimum number of hours a person works per week? What percentage of the people who work the minimum number of hours per week have a salary of more than 50K? 

In [21]:
# What is the minimum number of hours a person works per week (hours-per-week feature)?
min_work_hours = df['hours-per-week'].min()
min_work_hours, df['hours-per-week'].describe()

(np.int64(1),
 count    32561.000000
 mean        40.437456
 std         12.347429
 min          1.000000
 25%         40.000000
 50%         40.000000
 75%         45.000000
 max         99.000000
 Name: hours-per-week, dtype: float64)

In [22]:


# What percentage of the people who work the minimum number of hours per week have a salary of >50K?
min_workers = df[df['hours-per-week'] == min_work_hours]
num_min_workers = min_workers['hours-per-week']
num_min_workers



189      1
1036     1
1262     1
5590     1
5632     1
5766     1
5808     1
8447     1
9147     1
11451    1
19337    1
19750    1
20072    1
20909    1
22960    1
23536    1
24284    1
25078    1
29752    1
32525    1
Name: hours-per-week, dtype: int64

In [23]:
rich_min_workers = min_workers[min_workers['salary'].astype(str) == '>50K']
rich_min_workers['salary'].value_counts()

salary
>50K    2
Name: count, dtype: int64

In [24]:


# What percentage of the people who work the minimum number of hours per week have a salary of >50K?
rich_percentage = (len(rich_min_workers)/len(min_workers))*100
rich_percentage



10.0

In [25]:
# What is the minimum number of hours a person works per week (hours-per-week feature)?
min_work_hours

np.int64(1)

In [26]:
# What percentage of the people who work the minimum number of hours per week have a salary of >50K?
number_of_min_workers = len(num_min_workers)
rich_percentageof_min_workers = rich_percentage
number_of_min_workers, rich_percentageof_min_workers

(20, 10.0)

In [27]:
# Identify the most popular occupation for those who earn >50K in India
most_popular_occupation = df.groupby([df.occupation, df['native-country'][df['native-country']== 'India'], df.salary[df.salary == '>50K']])['occupation'].size()
top_IN_occupation = most_popular_occupation.idxmax()[0]
top_IN_occupation

'Prof-specialty'